In [ ]:
import os
from pathlib import Path

import chardet
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

plt.style.use(["science", "no-latex", "notebook", "grid"])
plt.rcParams["text.usetex"] = False

# Определяем текущую директорию и устанавливаем корень проекта
current_directory = Path().resolve()
os.chdir(current_directory.parent.parent)

# Создаем каталог для сохранения gif-анимаций
gif_dir = Path("docs/model_gifs")
gif_dir.mkdir(parents=True, exist_ok=True)

# Загрузка данных
file_path = current_directory / "conversion_df.csv"
if not file_path.exists():
    raise Exception(f"File {file_path} does not exist.")

with open(file_path, "rb") as f:
    result = chardet.detect(f.read())
file_encoding = result["encoding"]

df = pd.read_csv(file_path, encoding=file_encoding, sep=",")

# Подготовка данных: выбираем столбцы реакции и рассчитываем их разности
reaction_n = "reaction_0"
reaction_cols = [
    f"rate_3_diff_{reaction_n}_conversion",
    f"rate_5_diff_{reaction_n}_conversion",
    f"rate_10_diff_{reaction_n}_conversion",
]
_df = df[reaction_cols].copy()
dadt_data = [_df[col].diff() for col in _df]
reaction_df = pd.DataFrame(dadt_data).T.dropna()
reaction_df["temperature"] = df["temperature"] + 273.15
reaction_df.columns = ["3", "5", "10", "temperature"]

# Вычисляем кривую конверсии как кумулятивную сумму столбца "3"
conversion = reaction_df["3"].cumsum()
temperature = reaction_df["temperature"]

def r2_score(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - ss_res / ss_tot

# Импорт моделей (предполагается, что они импортируются из app_settings)
from src.core.app_settings import NUC_MODELS_TABLE

R = 8.314  # Газовая постоянная

# Итерируем по всем моделям из NUC_MODELS_TABLE
for model_name, model_info in NUC_MODELS_TABLE.items():

    differential_func = model_info["differential_form"]

    # Списки для накопления значений Ea, ln(A) и R² для построения динамики
    Ea_list = []
    lnA_list = []
    r2_list = []
    frame_files = []  # для хранения путей к сохраненным кадрам

    for Ea in tqdm(range(30000, 250001, 5000), desc=f"Обработка модели {model_name}"):
        try:
            # Вычисляем скорость реакции с учетом текущего Ea
            k = np.exp(-Ea / (R * temperature))
            func_val = k * differential_func(1 - conversion)

            # Расчет предэкспоненциального множителя A
            A = conversion.iloc[-1] / func_val.cumsum().iloc[-1]
            model_fit = func_val * A

            Ea_list.append(Ea)
            lnA_list.append(np.log(A))

            # Вычисляем R² между экспериментальной кривой (conversion) и моделью (кумулятивная модель)
            cum_model_fit = model_fit.cumsum()
            r2 = r2_score(conversion, cum_model_fit)
            r2_list.append(r2)

            # Построение графиков: 2x2 subplot (аналог старого кода)
            fig, axs = plt.subplots(2, 2, figsize=(12, 12))

            # График 1: Модель vs Эксперимент
            axs[0, 0].plot(temperature, model_fit, label="Модель")
            axs[0, 0].plot(temperature, conversion.diff(), label="Эксперимент")
            axs[0, 0].set_title(f"{model_name}: Модель vs Эксперимент")
            axs[0, 0].text(0.05, 0.95, f"ln(A): {np.round(np.log(A), 2)}\nEa: {Ea}",
                           transform=axs[0, 0].transAxes, va="top", fontsize=12)
            axs[0, 0].set_xlabel("Температура, К")
            axs[0, 0].set_ylabel("Скорость реакции")

            # График 2: Кумулятивные значения модели vs эксперимент
            axs[0, 1].plot(temperature, cum_model_fit, label="Модель")
            axs[0, 1].plot(temperature, conversion, label="Эксперимент")
            axs[0, 1].set_title("Конверсия от Т")
            axs[0, 1].set_xlabel("Температура, К")
            axs[0, 1].set_ylabel("Конверсия")

            # График 3: ln(A) в зависимости от Ea
            axs[1, 0].plot(Ea_list, lnA_list, label="ln(A) vs Ea")
            axs[1, 0].set_title("Зависимость ln(A) от Ea")
            axs[1, 0].set_xlabel("Ea, Дж/моль")
            axs[1, 0].set_ylabel("ln(A)")

            # График 4: R² в зависимости от Ea
            axs[1, 1].plot(Ea_list, r2_list, label="R² vs Ea")
            axs[1, 1].set_title("R² от Ea")
            axs[1, 1].set_xlabel("Ea, Дж/моль")
            axs[1, 1].set_ylabel("R²")

            plt.tight_layout()

            # Сохраняем текущий кадр в папку docs/model_gifs
            safe_model_name = model_name.replace("/", "_")
            frame_path = gif_dir / f"{safe_model_name}_Ea_{Ea}.png"
            plt.savefig(frame_path)
            frame_files.append(frame_path)

            plt.close(fig)

        except Exception as e:
            print(f"Ошибка для модели {model_name} при Ea={Ea}: {e}")
            break

    # Если кадры были созданы, формируем gif-анимацию
    if frame_files:
        frames = []
        for frame in frame_files:
            with Image.open(frame) as img:
                frames.append(img.copy())
        gif_path = gif_dir / f"{safe_model_name}.gif"
        frames[0].save(fp=gif_path, format="GIF", append_images=frames[1:],
                       save_all=True, duration=60, loop=0)
        # Удаляем временные png-файлы
        for frame in frame_files:
            os.remove(frame)


Обработка модели B1: 100%|██████████| 45/45 [00:43<00:00,  1.04it/s]
